In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [14]:
data = pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)



In [15]:
def create_model(neurons=32,layers=1):
    model = Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [16]:
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)


In [19]:
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [20]:
grid = GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3,verbose=1)
grid_result = grid.fit(X_train,y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/roronoazoro/Desktop/demo-alml/ChurnModelling-ANN/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/roronoazoro/Desktop/demo-alml/ChurnModelling-ANN/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/roronoazoro/Desktop/demo-alml/ChurnModelling-ANN/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/roronoazoro/Desktop/demo-alml/ChurnModelling-ANN/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2026-01-21 16:38:32.315453: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2026-01-21 16:38:32.320710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2026-01-21 16:38:32.348580: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2026-01-21 16:38:32.367073: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2026-01-21 16:38:32.391302: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2026-01-21 16:38:32.391446: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2026-01-21 16:38:32.416024: I tensorflow/core/grappler/optimizers/cust

167/167 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.6915 - loss: 0.6153
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.7035 - loss: 0.5895
167/167 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.6372 - loss: 0.6900
Epoch 2/50
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.7328 - loss: 0.5474
167/167 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.7538 - loss: 0.5252
  1/167 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step - accuracy: 0.7188 - loss: 0.5206Epoch 2/50
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.7596 - loss: 0.5209
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.7182 - loss: 0.5738
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.7253 - loss: 0.5617
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.7958 - loss: 0.4633
Epoch 3/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8112 - loss: 0.4390
Epoch 3/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 

...

167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.7762 - loss: 0.5924
Epoch 35/100
 97/167 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.8001 - loss: 0.4645Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.7979 - loss: 0.4739
127/167 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.8009 - loss: 0.4639Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.7643 - loss: 0.7276
Epoch 30/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.8076 - loss: 0.4576
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.7995 - loss: 0.4564
Epoch 3/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.7727 - loss: 0.6593
Epoch 26/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.7853 - loss: 0.5671
Epoch 34/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.7904 - loss: 0.5855
Epoch 36/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.7917 - loss: 0.4964
Epoch 6/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step

/Users/roronoazoro/Desktop/demo-alml/ChurnModelling-ANN/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/roronoazoro/Desktop/demo-alml/ChurnModelling-ANN/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2026-01-21 17:17:46.491209: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2026-01-21 17:17:46.515168: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2026-01-21 17:17:46.518415: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2026-01-21 17:17:46.518442: I 

Epoch 1/100


2026-01-21 17:17:48.716953: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7243 - loss: 0.5535
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8091 - loss: 0.4393
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8110 - loss: 0.4353
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8111 - loss: 0.4349
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8121 - loss: 0.4356
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8099 - loss: 0.4340
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8100 - loss: 0.4343
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8119 - loss: 0.4336
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8098 - loss: 0.4339
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8109 - loss: 0.4338
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8098 - loss: 0.4338
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step